In [1]:
import numpy as np, pandas as pd, os, pickle
from natsort import natsorted
from scipy.signal import find_peaks
from datetime import timedelta
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output


print('Header files imported......')

Header files imported......


In [19]:
def plot_sensor_active_periods(data_subject_2, environmental_sensors, st, et):
    if st != None and et!=None:
        print('I came inside......')
        start_date = st #pd.to_datetime('2023-07-21 00:00:00')
        end_date = et #pd.to_datetime('2023-07-26 00:00:00')

    df_combined_non_temp_humidity = []
    df_combined_temp_humidity = {}
    for sens in range(len(environmental_sensors)):
        sensor_name = environmental_sensors[sens]
        
        print(sens, sensor_name)
        if sensor_name in ['Shower_Hum_Temp_humidity.csv','Shower_Hum_Temp_temp.csv','Stove_Hum_Temp_humidity.csv','Stove_Hum_Temp_temp.csv']:
            df_sensor = data_subject_2[sensor_name][0]
            df_sensor['ts_datetime'] = pd.to_datetime(df_sensor['ts_datetime'])
            if st!=None and et!=None:
                filtered_df = df_sensor[(df_sensor['ts_datetime'] >= start_date) & (df_sensor['ts_datetime'] <= end_date)]
            else:
                filtered_df = df_sensor.copy()
            sub_df_sensor = filtered_df[['sensor_id', 'ts_datetime', 'sensor_status']]
            df_combined_temp_humidity[sensor_name] = sub_df_sensor
            sub_df_sensor = pd.DataFrame()
        else:
            df_sensor = data_subject_2[sensor_name][1]
            df_sensor['ts_on'] = pd.to_datetime(df_sensor['ts_on'])
            df_sensor['ts_off'] = pd.to_datetime(df_sensor['ts_off'])
            if st != None and et!=None:
                filtered_df = df_sensor[(df_sensor['ts_on'] >= start_date) & (df_sensor['ts_on'] <= end_date)]
            else:
                filtered_df = df_sensor.copy()
            sub_df_sensor = filtered_df[['sensor_id', 'ts_on', 'ts_off']]
            df_combined_non_temp_humidity.append(sub_df_sensor)
            sub_df_sensor = pd.DataFrame()

    df_combined_non_temp_humidity = pd.concat(df_combined_non_temp_humidity, ignore_index=True)
    df_combined_non_temp_humidity = df_combined_non_temp_humidity.sort_values(by=['sensor_id'], ascending=True)

    # clear_output(wait=True)

    fig = make_subplots(
        rows=3, 
        cols=1, 
        shared_xaxes=True, 
        row_heights=[0.1, 0.1, 0.4],
        vertical_spacing = 0.01
    ) 

    fig.add_trace(
        go.Scatter(
            x=df_combined_temp_humidity['Stove_Hum_Temp_temp.csv'].reset_index().ts_datetime,
            y=df_combined_temp_humidity['Stove_Hum_Temp_temp.csv'].reset_index().sensor_status, 
            mode='lines', 
            name='Temperature', 
            line=dict(color='red')
        ), 
        row=1, 
        col=1
    )

    fig.add_trace(
        go.Scatter(
            x=df_combined_temp_humidity['Stove_Hum_Temp_humidity.csv'].reset_index().ts_datetime,
            y=df_combined_temp_humidity['Stove_Hum_Temp_humidity.csv'].reset_index().sensor_status,  
            mode='lines', 
            name='Humidity', 
            line=dict(color='blue')
        ), 
        row=2, 
        col=1
    )
    fig1 = go.Figure(px.timeline(df_combined_non_temp_humidity, x_start='ts_on', x_end='ts_off', y='sensor_id', color='sensor_id',labels={'sensor_id': 'Sensor ID'}))
    for i in range(len(fig1.data)):
        fig.append_trace(fig1.data[i],row=3,col=1)


    fig.update_layout(barmode='overlay')
    fig.update_yaxes(fixedrange=True)
    fig.update_layout(height=500) 
    fig.update_xaxes(type='date')
    fig.show()        


In [20]:
with open('/home/hubble/work/serenade/src/subject_data_sepetember2023_may2024_2hours.pkl', 'rb') as file:
    data = pickle.load(file)

In [21]:
subjects = list(data.keys())
print(subjects)

['subject_1', 'subject_2', 'subject_3', 'subject_4', 'subject_5', 'subject_7', 'subject_8']


In [22]:

for sbj in range(1,len(subjects)):
    subject = subjects[sbj]
    data_subject_2 = data[subject]
    environmental_sensors = list(data_subject_2.keys())
    st = pd.to_datetime('2024-05-01 00:00:00')
    et = pd.to_datetime('2024-05-08 23:59:59')
    plot_sensor_active_periods(data_subject_2, environmental_sensors, st, et)

I came inside......
0 Dishes_Silverware.csv
1 FoodStorage.csv
2 HouseEntrance.csv
3 Medicines.csv
4 MotionBathroom.csv
5 MotionBedroom.csv
6 MotionKitchen.csv
7 MotionLivingroom.csv
8 MotionOtherroom.csv
9 PlugTvHall.csv
10 Refrigerator.csv
11 Shower_Hum_Temp_humidity.csv
12 Shower_Hum_Temp_temp.csv
13 Stove_Hum_Temp_humidity.csv
14 Stove_Hum_Temp_temp.csv


KeyError: 'sensor_status'